In [52]:
import pandas as pd

In [53]:
# File path

data_path = r"C:\Users\...\raw_data"


In [54]:
# importing data files useing pandas

price = pd.read_csv(f"{data_path}/all_price.csv")
income = pd.read_csv(f"{data_path}/all_income.csv")
bs = pd.read_csv(f"{data_path}/all_balancesheet.csv")
cf = pd.read_csv(f"{data_path}/all_cashflow.csv")
earn = pd.read_csv(f"{data_path}/all_earnings.csv")

price.head(), income.head(), bs.head(), cf.head(), earn.head()


(        symbol        date        open        high         low       close  \
 0  RELIANCE.NS  2020-11-15  929.876038  930.060608  920.853699  924.061096   
 1  RELIANCE.NS  2020-11-17  962.227173  962.227173  916.077148  919.884521   
 2  RELIANCE.NS  2020-11-18  919.884521  925.999451  904.585815  917.092468   
 3  RELIANCE.NS  2020-11-19  917.092468  927.822327  908.323975  910.608398   
 4  RELIANCE.NS  2020-11-20  911.462158  915.523376  874.542175  876.618958   
 
      volume    adjclose  dividends  splits  
 0   5226220  907.905396        0.0     0.0  
 1  46542562  903.801880        0.0     0.0  
 2  30402290  901.058533        0.0     0.0  
 3  27796343  894.687866        0.0     0.0  
 4  57471248  861.292725        0.0     0.0  ,
           symbol    asOfDate periodType currencyCode  Amortization  \
 0  ADANIPORTS.NS  2021-03-31        12M          INR           NaN   
 1  ADANIPORTS.NS  2022-03-31        12M          INR           NaN   
 2  ADANIPORTS.NS  2023-03-31     

####  CLEAN INCOME STATEMENT

In [55]:
income_clean = income.copy()

# Extracting Year from asOfDate
income_clean["year"] = pd.to_datetime(income_clean["asOfDate"]).dt.year

# Taking only needed columns and renaming
income_clean = income_clean[[
    "symbol", "year",
    "TotalRevenue",
    "CostOfRevenue",
    "GrossProfit",
    "OperatingIncome",
    "NetIncome",
]]

income_clean.columns = [
    "ticker", "year",
    "revenue",
    "cost_of_revenue",
    "gross_profit",
    "operating_income",
    "net_income"
]

income_clean.head()

,ticker,year,revenue,cost_of_revenue,gross_profit,operating_income,net_income
0,ADANIPORTS.NS,2021,NaN,NaN,NaN,NaN,NaN
1,ADANIPORTS.NS,2022,1.697908e+11,7.069400e+10,9.909680e+10,7.445870e+10,4.886030e+10
2,ADANIPORTS.NS,2023,2.085191e+11,9.061760e+10,1.179015e+11,9.512750e+10,5.308850e+10
3,ADANIPORTS.NS,2023,NaN,NaN,NaN,NaN,NaN
4,ADANIPORTS.NS,2023,NaN,NaN,NaN,NaN,NaN


##### Income data cleaning

In [56]:
# Cleaning missing values
income_clean = income_clean.dropna(subset=["revenue", "cost_of_revenue", "gross_profit", "operating_income", "net_income"],
                                 how = 'all'
)

# Cleaning duplicates
income_clean = income_clean.drop_duplicates(subset=["ticker","year"],keep ='first'
                                           )

# Only keeping revenue greater than Zero
income_clean = income_clean[income_clean["revenue"] > 0]

# Converting all numerical cloumn data type to float
num_cols = ["revenue", "cost_of_revenue", "gross_profit", "operating_income", "net_income"]
income_clean[num_cols] = income_clean[num_cols].astype(float)

# Sorting all
income_clean =income_clean.sort_values(["ticker","year"]).reset_index(drop= True)


#### CLEAN BALANCE SHEET

In [57]:
balance_clean = bs.copy()

# Extract year from date FIRST
balance_clean["year"] = pd.to_datetime(balance_clean["asOfDate"]).dt.year

# Select columns WITHOUT overwriting year incorrectly
balance_clean = balance_clean[[
    "symbol",
    "year",  # <-- keep correct year
    "TotalAssets",
    "TotalLiabilitiesNetMinorityInterest",
    "TotalEquityGrossMinorityInterest",
    "CurrentAssets"
]]

# Rename properly
balance_clean.columns = [
    "ticker",
    "year",
    "total_assets",
    "total_liabilities",
    "equity",
    "current_assets"
]

# Drop missing rows ONLY if all 3 are missing
balance_clean = balance_clean.dropna(subset=["total_assets", "total_liabilities", "equity"], how="all")

# Remove duplicates
balance_clean = balance_clean.drop_duplicates(subset=["ticker", "year"], keep="first")

# Convert numbers
num_cols = ["total_assets", "total_liabilities", "equity", "current_assets"]
balance_clean[num_cols] = balance_clean[num_cols].astype(float)

# Filter valid values
balance_clean = balance_clean[(balance_clean["total_assets"] > 0)]

# Sort
balance_clean = balance_clean.sort_values(["ticker", "year"]).reset_index(drop=True)
balance_clean.head()

,ticker,year,total_assets,total_liabilities,equity,current_assets
0,ADANIPORTS.NS,2022,9.968582e+11,5.730483e+11,4.238099e+11,1.808842e+11
1,ADANIPORTS.NS,2023,1.147631e+12,6.784611e+11,4.691698e+11,1.759639e+11
2,ADANIPORTS.NS,2024,1.189179e+12,6.437487e+11,5.454300e+11,1.747236e+11
3,ADANIPORTS.NS,2025,1.353322e+12,7.035889e+11,6.497334e+11,1.894412e+11
4,BAJFINANCE.NS,2022,2.125059e+12,1.687932e+12,4.371269e+11,NaN


In [47]:
balance_clean = bs.copy()
balance_clean["year"] = pd.to_datetime(balance_clean["asOfDate"]).dt.year

# Selecting useful columns
balance_cols = [
    "symbol", "year",
    "TotalAssets",
    "TotalLiabilitiesNetMinorityInterest",
    "TotalEquityGrossMinorityInterest",
    "CurrentAssets",
    "CurrentLiabilities"
]

# Keep only columns that actually exist in the dataset
balance_cols = [c for c in balance_cols if c in bs.columns]

balance_clean = balance_clean[balance_cols]

balance_clean.columns = [
    "ticker", "year", 
    "total_assets", "total_liabilities", "equity",
    "current_assets", "current_liabilities"
][:len(balance_cols)]  



    

##### Balance sheet data cleaning

In [48]:
# Clean missing values (at least total assets must exist)
balance_clean = balance_clean.dropna(subset=["total_assets"], how="any")

# Remove duplicates
balance_clean = balance_clean.drop_duplicates(subset=["ticker", "year"], keep="first")

# Convert numeric
numeric = ["total_assets", "total_liabilities", "equity",
           "current_assets", "current_liabilities"]
for col in numeric:
    if col in balance_clean.columns:
        balance_clean[col] = pd.to_numeric(balance_clean[col], errors='coerce')

# Remove invalid or zero assets
balance_clean = balance_clean[balance_clean["total_assets"] > 0]

# Sort & reset index
balance_clean = balance_clean.sort_values(["ticker", "year"]).reset_index(drop=True)

balance_clean.head()


,ticker,year,total_assets,total_liabilities,equity,current_assets
0,ADANIPORTS.NS,9.968582e+11,5.730483e+11,4.238099e+11,1.808842e+11,1.136871e+11
1,ADANIPORTS.NS,1.147631e+12,6.784611e+11,4.691698e+11,1.759639e+11,1.253012e+11
2,ADANIPORTS.NS,1.189179e+12,6.437487e+11,5.454300e+11,1.747236e+11,1.646294e+11
3,ADANIPORTS.NS,1.353322e+12,7.035889e+11,6.497334e+11,1.894412e+11,2.098599e+11
4,BAJFINANCE.NS,2.125059e+12,1.687932e+12,4.371269e+11,NaN,NaN


####  CLEAN CASH FLOW

In [58]:
cf_clean = cf.copy()
cf_clean["year"] = pd.to_datetime(cf["asOfDate"]).dt.year

cf_clean = cf_clean[[
    "symbol",
    "year",
    "OperatingCashFlow",
    "CapitalExpenditure",
    "FreeCashFlow",
]]

cf_clean.columns = [
    "ticker", 
    "year",
    "operating_cashflow",
    "capex",
    "free_cashflow"
]

cf_clean.head()


,ticker,year,operating_cashflow,capex,free_cashflow
0,ADANIPORTS.NS,2020,7.248690e+10,NaN,3.627280e+10
1,ADANIPORTS.NS,2021,NaN,NaN,NaN
2,ADANIPORTS.NS,2022,1.042014e+11,-3.813700e+10,6.606440e+10
3,ADANIPORTS.NS,2023,1.189950e+11,-9.141040e+10,2.758460e+10
4,ADANIPORTS.NS,2024,1.501758e+11,-7.416300e+10,7.601280e+10


##### Cash flow data cleaning

In [59]:
### CLEAN CASHFLOW 

cf_clean = cf_clean.copy()

# Keeping only rows that have operating cash flow 
cf_clean = cf_clean.dropna(subset=["operating_cashflow"], how="any")

# Converting numeric columns
num_cols = ["operating_cashflow", "capex", "free_cashflow"]
cf_clean[num_cols] = cf_clean[num_cols].astype(float)

# Sorting properly
cf_clean = cf_clean.sort_values(["ticker", "year"]).reset_index(drop=True)

cf_clean.head()


,ticker,year,operating_cashflow,capex,free_cashflow
0,ADANIPORTS.NS,2020,7.248690e+10,NaN,3.627280e+10
1,ADANIPORTS.NS,2022,1.042014e+11,-3.813700e+10,6.606440e+10
2,ADANIPORTS.NS,2023,1.189950e+11,-9.141040e+10,2.758460e+10
3,ADANIPORTS.NS,2024,1.501758e+11,-7.416300e+10,7.601280e+10
4,ADANIPORTS.NS,2025,1.722628e+11,-8.048920e+10,9.177360e+10


####  CLEAN EARINGS (EPS)

In [60]:
# Adding ticker (comes from 'symbol')
income["Ticker"] = income["symbol"]

def ticker_to_name(t):
    # remove .NS, replace dashes/underscores, split camelcase-ish parts, then title-case
    s = t.replace(".NS","").replace("_"," ").replace("-"," ")
    return s.title()

income["Company"] = income["symbol"].map(lambda x: ticker_to_name(x))

# Building EPS dataset

eps_rows = []

for idx, row in income.iterrows():

    eps_rows.append({
        "ticker": row["Ticker"],
        "company": row["Company"],
        "period": row["asOfDate"],
        "eps_basic": row["BasicEPS"],
        "eps_diluted": row["DilutedEPS"]
    })

eps_df = pd.DataFrame(eps_rows)

# Extracting  year
eps_df["year"] = eps_df["period"].str[:4].astype(int)

# Drop duplicates for safety
eps_df.drop_duplicates(subset=["ticker", "period"], inplace=True)

eps_df.head()



   

,ticker,company,period,eps_basic,eps_diluted,year
0,ADANIPORTS.NS,Adaniports,2021-03-31,NaN,NaN,2021
1,ADANIPORTS.NS,Adaniports,2022-03-31,22.62,22.62,2022
2,ADANIPORTS.NS,Adaniports,2023-03-31,24.58,24.58,2023
3,ADANIPORTS.NS,Adaniports,2023-09-30,29.33,29.33,2023
4,ADANIPORTS.NS,Adaniports,2023-12-31,33.46,33.46,2023


In [62]:
# only useing data from 2018 to 2024
income_clean = income_clean[(income_clean["year"] >= 2018) & (income_clean["year"] <= 2024)]
balance_clean = balance_clean[(balance_clean["year"] >= 2018) & (balance_clean["year"] <= 2024)]
cf_clean = cf_clean[(cf_clean["year"] >= 2018) & (cf_clean["year"] <= 2024)]
eps_df = eps_df[(eps_df["year"] >= 2018) & (eps_df["year"] <= 2024)]


In [63]:
income_clean.to_csv(f"C:\\Users\...\clean_income.csv", index=False)
balance_clean.to_csv(f"C:\\Users\...\clean_balance.csv", index=False)
cf_clean.to_csv(f"C:\...\clean_cashflow.csv", index=False)
eps_df.to_csv(f"C:\...\clean_eps.csv", index=False)


In [61]:
print(income_clean['year'].unique())
print(balance_clean['year'].unique())
print(cf_clean['year'].unique())
print(eps_df['year'].unique())


[2022 2023 2024 2025]
[2022 2023 2024 2025]
[2020 2022 2023 2024 2025]
[2021 2022 2023 2024 2025 2020 2017]
